In [2]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from torch import optim
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import math
import copy

# from torchvision.models import resnet50, ResNet50_Weights

torch.manual_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self, max_growth):
        super(LeNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc1 = nn.Linear(400,500+max_growth)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(500+max_growth,100+max_growth)
        self.relu1 = nn.ReLU()
        self.fc3 = nn.Linear(100+max_growth,10)

        #Custom Functionality
        self.max_growth = max_growth
        previous_module = self.parameters_to_prune()[0][0]

        for module, name in self.parameters_to_prune()[1:]:

          #Prune incoming weights (Previous layers outgoing weights)
          if previous_module != None:
            mask = torch.ones(np.shape(previous_module.weight))
            mask[-max_growth:,:] = 0
            prune.custom_from_mask(previous_module, name, mask)
            
          #Prune outgoing weights
          mask = torch.ones(np.shape(module.weight))
          mask[:,-max_growth:] = 0
          prune.custom_from_mask(module, name, mask)

          previous_module = module  #save layer for next iteration
        

    def parameters_to_prune(self):
        return ((self.fc1, 'weight'),
                (self.fc2, 'weight'),
                (self.fc3, 'weight'),) 
    #End Custom Functionality

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu1(out)
        out = self.fc3(out)
        return out

In [4]:
# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001

#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.CIFAR10(root = '/Users/dimademler/Desktop/me-Programming/pytorch',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.CIFAR10(root = '/Users/dimademler/Desktop/me-Programming/pytorch',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

augmented_train_dataset = torchvision.datasets.CIFAR10(root = '/Users/dimademler/Desktop/me-Programming/pytorch',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.RandomVerticalFlip(p=1),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)

augmented_train_loader = torch.utils.data.DataLoader(dataset = augmented_train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

augmented_test_dataset = torchvision.datasets.CIFAR10(root = '/Users/dimademler/Desktop/me-Programming/pytorch',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.RandomVerticalFlip(p=1),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


augmented_test_loader = torch.utils.data.DataLoader(dataset = augmented_test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

def train(model,train_loader,test_loader,num_epochs,optimizer):
  total_step = len(train_loader)
  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):  
          images = images.to(device)
          labels = labels.to(device)
          
          #Forward pass
          outputs = model(images)
          loss = cost(outputs, labels)
            
          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
              
          if (i+1) % 400 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                        .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
  
def test(model, test_loader):
    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):  
            test_output = model(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))

    print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    return accuracy

PermissionError: [Errno 13] Permission denied: '/Users'

In [5]:
#All in One Method to Find Lottery Tickets given a model and dataset, saves models in shared drive
def find_ticket(model, name, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 10, learning_rate = .001, prune_amount = .2, k = 3):
  
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  #If training has already been saved
  try:
    model.load_state_dict(torch.load('/Users/dimademler/Desktop/me-Programming/pytorch/Adaptive_ML/models_cifar_1/'+ name + '_RewindWeights' + '_' + str(k)))
  except:
    train(model, train_loader,test_loader,num_epochs = k, optimizer = optimizer)  #Save Kth epoch model
    torch.save(model.state_dict(), '/Users/dimademler/Desktop/me-Programming/pytorch/Adaptive_ML/models_cifar_1/' + name + '_RewindWeights' + '_' + str(k))

  model_rewind = LeNet(max_growth = model.max_growth) #Save as separate model so we can rewind our weights back to this
  model_rewind.load_state_dict(torch.load('/Users/dimademler/Desktop/me-Programming/pytorch/Adaptive_ML/models_cifar_1/'+ name + '_RewindWeights' + '_' + str(k)))

  train(model, train_loader,test_loader,num_epochs = num_epochs - k, optimizer = optimizer) #Finish off training
  accuracy = []

  for i in range(start_iter, end_iter): 
    #Prune
    prune.global_unstructured(model.parameters_to_prune(),pruning_method=prune.L1Unstructured,amount=prune_amount,)
    
    #Rewind Weights
    for idx, (module, _) in enumerate(model.parameters_to_prune()):
      with torch.no_grad():
        module_rewind = model_rewind.parameters_to_prune()[idx][0]
        module.weight_orig.copy_(module_rewind.weight)

    print(
    "Global Sparsity: {:.2f}%".format(
        100. * (float(torch.sum(model.fc3.weight == 0)) + float(torch.sum(model.fc1.weight == 0)) + float(torch.sum(model.fc2.weight == 0)))
        / (float(model.fc3.weight.nelement()) + float(model.fc1.weight.nelement()) + float(model.fc2.weight.nelement()))
    )
    )
    
    #Train
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train(model, train_loader,test_loader,num_epochs = num_epochs, optimizer = optimizer)
    accuracy.append(test(model, test_loader))

    plt.plot(np.arange(len(accuracy)), accuracy)
    plt.show()
    print('Saving iteration ', str(i+1))
    torch.save(model.state_dict(), '/Users/dimademler/Desktop/me-Programming/pytorch/Adaptive_ML/models_cifar_1/' + name + '_iter' + str(i+1)) 

def grow(model, parameters_to_prune, amount):
  previous_module = parameters_to_prune[0][0]

  for module, name in parameters_to_prune[1:]:
    mask = module.get_buffer('weight_mask').data
    #Pick Suitable Locations
    omega = []
    for idx, val in enumerate(torch.sum(mask, dim = 0)): 
      if val == 0:
        omega.append(idx)

    indices = np.random.choice(omega, size = min(len(omega), amount), replace = False)
    
    #Grow at these indices
    module.get_buffer('weight_mask')[:,indices] = 1
    prune.custom_from_mask(module,'weight', torch.ones(module.weight.size()))

    previous_module.get_buffer('weight_mask')[indices,:] = 1
    prune.custom_from_mask(previous_module,'weight', torch.ones(previous_module.weight.size()))  

In [6]:
#Generate Tickets

max_growth = 50
model = LeNet(max_growth = max_growth)
find_ticket(model, 'CIFAR10', train_loader, test_loader)

NameError: name 'train_loader' is not defined